#Kodo-python Getting Started

Welcome to the getting started ipython notebook for kodo-python.

This guide is intended for newcomers to the Kodo library. The guide will in tiny steps guide you through the creation and usage of both encoders and decoders.
Even though this guide focuses on the Python language bindings for Kodo - a similar API exists for other languages including C, C++ and Java.

##Importing Kodo

Before working with Kodo-python, you obviously need to have it installed and available. To ensure that's the case, try importing it:

In [1]:
# try importing the ``kodo`` module
try:
    import kodo
    print("Kodo imported successfully")
except ImportError:
    print("Unable to import kodo!")

Kodo imported successfully


If the import worked, you are ready to move on to the next step. Otherwise please (re)visit the README.rst for installation instructions.

## Creating an Encoder

In kodo, both encoders and decoders are created using factories. Doing so allows efficient memory management and reuse of various components and computations. 

Therefore, before creating an encoder, let's look at the encoder factories provided by the ``kodo`` module:

In [2]:
# print all members containing "Factory" and "Encoder"
print("\n".join([item for item in dir(kodo) if all([keyword in item for keyword in ["Factory", "Encoder"]])]))

FulcrumEncoderFactory
NoCodeEncoderFactory
PerpetualEncoderFactory
RLNCEncoderFactory


As seen from the output, several different encoder factories exist. Most of these have decoder factory counterparts.

The factory names refer to the encoding algorithm used.

For this walkthrough we pick the RLNC factory, i.e. the **``RLNC``**``EncoderFactory`` factory.

Note: *For this guide, any of the factories should work. For this reason I'll define the factory class as ``EncoderFactory``.*

In [3]:
# Store the RLNC encoder factory as EncoderFactory
EncoderFactory = kodo.RLNCEncoderFactory

By using python's ``help`` function, we can inspect the  ``EncoderFactory``'s constructor: 

In [4]:
# Get information about the encoder factory's __init__ function
help(EncoderFactory.__init__)

Help on method __init__:

__init__(...) unbound kodo.RLNCEncoderFactory method
    Factory constructor.
    
            :param field: The finite field to use.
            :param symbols: The number of symbols in a block.
            :param symbol_size: The size of a symbol in bytes.



From the documentation, we can see that we need to provide the ``field``, ``symbols`` and ``symbol_size`` parameters to create a factory.

The proper values to pick depends on the use case, we'll choose the binary field and the numbers 4 and 32 for the symbols and symbol_size, respectively.
These numbers would be very low for a real use case, but they serve us well for this example.

Let's create an encoder_factory:

In [5]:
field = kodo.field.binary
symbols = 4
symbol_size = 32

encoder_factory = EncoderFactory(field, symbols, symbol_size)

To see which methods are available for the encoder_factory, we can use python's ``dir`` function.

In [6]:
# Print all public members
print("\n".join([item for item in dir(encoder_factory) if not item.startswith("__")]))

build
set_symbol_size
set_symbols
symbol_size
symbols


The ``build`` method is used for creating encoders:

In [9]:
encoder = encoder_factory.build()

Fantastic, we've build our first encoder! Let's see what we can use it for:

In [10]:
# Print all public members
print("\n".join([item for item in dir(encoder) if not item.startswith("__")]))

block_size
in_systematic_phase
is_symbol_pivot
is_systematic_on
payload_size
rank
set_const_symbol
set_const_symbols
set_systematic_off
set_systematic_on
set_trace_callback
set_trace_off
set_trace_stdout
set_zone_prefix
symbol_size
symbols
write_payload


Let's inspect the state of our newly created encoder.

In [11]:
def print_encoder_state(encoder):
    print(
        "block_size: {}\n"
        "is_systematic_on: {}\n"
        "in_systematic_phase: {}\n"
        "payload_size: {}\n"
        "rank: {}\n"
        "symbol_size: {}\n"
        "symbols: {}".format(
            encoder.block_size(),
            encoder.is_systematic_on(),
            encoder.in_systematic_phase(),
            encoder.payload_size(),
            encoder.rank(),
            encoder.symbol_size(),
            encoder.symbols())
    )
print_encoder_state(encoder)

block_size: 128
is_systematic_on: True
in_systematic_phase: False
payload_size: 38
rank: 0
symbol_size: 32
symbols: 4


## Using the Encoder

We use the ``write_payload`` method to encode the data, but since we have yet to tell encoder what data to encode, we can't use it yet.
This can be seen from the encoder rank which is 0.

Let's create some data to encode:

In [12]:
data_in = bytearray(
    "The size of this data is exactly 128 bytes "
    "which means it will fit perfectly in a single generation. "
    "That is very lucky, indeed!"
)
print("Length of data string: {}".format(len(data_in)))

Length of data string: 128


Kodo uses Python bytearrays as data objects. Let's set the data to encode on the encoder.

In [13]:
encoder.set_const_symbols(data_in)

We should now be able to see how the state of the encoder has changed.

In [14]:
print_encoder_state(encoder)

block_size: 128
is_systematic_on: True
in_systematic_phase: True
payload_size: 38
rank: 4
symbol_size: 32
symbols: 4


Notice how the rank is now equal to the number of symbols:

In [16]:
encoder.rank() == symbols

True

We can only encode packets if the rank is ``> 0``.

Let's encode some packets using the ``write_payload`` method:

In [17]:
packet1 = encoder.write_payload()
packet2 = encoder.write_payload()
packet3 = encoder.write_payload()
packet4 = encoder.write_payload()

print(
    "packet1: {}\n"
    "packet2: {}\n"
    "packet3: {}\n"
    "packet4: {}\n".format(
        packet1,
        packet2,
        packet3,
        packet4,
    )
)

packet1: �    The size of this data is exactly
packet2: �    128 bytes which means it will f
packet3: �   it perfectly in a single generat
packet4: �   ion. That is very lucky, indeed!



Notice how all the packets are prefixed with ``�`` - this is python trying to print the packet header containing the symbol id.

The reason why the contents of the packets are readable is that the encoder is in systematic phase. Systematic means that the encoder keeps each symbol uncoded in the first iteration.

Because we've set the generation size to be 4 symbols, and we've created 4 packets - the encoder is no longer in systematic phase:  

In [18]:
encoder.in_systematic_phase()

False

This means that any subsequent packets will be encoded.

In [19]:
packet5 = encoder.write_payload()
print("packet5: {}".format(packet5))

packet5:  � *|feDwpr'%}hvh:8mz}c5 t.|cl{%3


Since the encoding is random, the data could still be uncoded, it will however most likely be unreadable.

## Creating a Decoder

Let's create a decoder factory, a decoder and an output buffer so that we can decode our newly generated packets:

In [20]:
decoder_factory = kodo.RLNCDecoderFactory(field, symbols, symbol_size)
decoder = decoder_factory.build()
data_out = bytearray(decoder.block_size())
decoder.set_mutable_symbols(data_out)

Let's investigate which methods are available for the decoder:

In [21]:
# Print all public members
print("\n".join([item for item in dir(decoder) if not item.startswith("__")]))

block_size
is_complete
is_symbol_missing
is_symbol_partially_decoded
is_symbol_pivot
is_symbol_uncoded
payload_size
rank
read_payload
set_mutable_symbol
set_mutable_symbols
set_trace_callback
set_trace_stdout
set_zone_prefix
symbol_size
symbols
symbols_missing
symbols_partially_decoded
symbols_uncoded
write_payload


As seen from the output, the encoder and decoder share a few methods. Most of these have the same meaning.
Let's inspect the state of our newly created decoder.

In [22]:
def print_decoder_state(decoder):
    print(
        "block_size: {}\n"
        "is_complete: {}\n"
        "payload_size: {}\n"
        "rank: {}\n"
        "symbol_size: {}\n"
        "symbols: {}\n"
        "symbols_uncoded: {}\n".format(
            decoder.block_size(),
            decoder.is_complete(),
            decoder.payload_size(),
            decoder.rank(),
            decoder.symbol_size(),
            decoder.symbols(),
            decoder.symbols_uncoded())
    )
print_decoder_state(decoder)

block_size: 128
is_complete: False
payload_size: 38
rank: 0
symbol_size: 32
symbols: 4
symbols_uncoded: 0



What's probably the most interesting here is the rank. The rank corresponds to the number of innovative packets received.

## Using the Decoder

If we read one of our previously generated packets, we should see the rank increase:

In [23]:
decoder.read_payload(packet1)
decoder.rank()

1

And it does.

We can now try to read the 5th packet, and see what it does to the state. The unique thing about the 5th packet, is that it's the only one which has been encoded, due to our encoder being systematic.

In [24]:
decoder.read_payload(packet5)
print_decoder_state(decoder)

block_size: 128
is_complete: False
payload_size: 38
rank: 2
symbol_size: 32
symbols: 4
symbols_uncoded: 1



The rank has increased to 2! This means that we've read two (innovative) packets. If we extract the current data in the decoder we get the following output:

In [25]:
data_out.replace('\x00', '_')

"The size of this data is exactly *|feDwpr'%}hvh:8mz}c\x7f5 t.|cl{%3________________________________________________________________"

Notice that the first part of the string is readable. Depending on the encoding of the 5th packet other parts of the string may or may not be readable.

Note that the rank may only increase by one when reading a packet.

If we start feeding the decoder new data, we will at one point have a complete generation:

In [27]:
while not decoder.is_complete():
    decoder.read_payload(encoder.write_payload())
    print(decoder.rank())

3
3
3
3
3
4


And when the decoding is complete, the original bytearray should be reconstructed in data_out:

In [28]:
print(data_out)
data_out == data_in

The size of this data is exactly 128 bytes which means it will fit perfectly in a single generation. That is very lucky, indeed!


True

Hurray, it worked!

For more information and inspiration please look through some of the many examples.